In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
# import quantstats as qs







In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

closing_price1 = pd.read_excel('fechamento_limpo.xlsx')
print('1')
quality1 = pd.read_excel('base_quality_limpa.xlsx')
print('1')
low_size1 = pd.read_excel('base_low_size_limpa.xlsx')
print('1')
value1 = pd.read_excel('value_limpo.xlsx')


1
1
1


In [3]:
low_size1["Data"] = pd.to_datetime(low_size1["Data"])
low_size1 = low_size1.set_index("Data")

quality1["Data"] = pd.to_datetime(quality1["Data"])
quality1 = quality1.set_index("Data")

value1["Data"] = pd.to_datetime(value1["Data"])
value1 = value1.set_index("Data")

closing_price1["Data"] = pd.to_datetime(closing_price1["Data"])
closing_price1 = closing_price1.set_index("Data")

In [4]:
dados_ineg1 = pd.read_excel('ineg_limpo (3).xlsx')
dados_ineg1['Data'] = pd.to_datetime(dados_ineg1['Data'])
dados_ineg1 = dados_ineg1.set_index('Data')

In [ ]:

initial_date = pd.Timestamp(dt.datetime(2000,1,24))
final_date =pd.Timestamp(dt.datetime(2024,12,31))


low_size2 = low_size1[(low_size1.index >= initial_date) & (low_size1.index <=final_date)]
closing_price2 = closing_price1[(closing_price1.index >= initial_date) & (closing_price1.index <=final_date)]
quality2 = quality1[(quality1.index >= initial_date) & (quality1.index <=final_date)]
value2 = value1[(value1.index >= initial_date) & (value1.index <=final_date)]
dados_ineg2 = dados_ineg1[(dados_ineg1.index >= initial_date) & (dados_ineg1.index <=final_date)]

#-------------------------------------------------------------------------------

# Força o índice para datetime corretamente
closing_price = closing_price1.copy()
closing_price.index = pd.to_datetime(closing_price.index, errors='coerce')
closing_price = closing_price[closing_price.index.notna()]  # remove datas inválidas


low_size = low_size1.copy()
low_size.index = pd.to_datetime(low_size.index, errors='coerce')
low_size = low_size[low_size.index.notna()]


quality = quality1.copy()
quality.index = pd.to_datetime(quality.index, errors='coerce')
quality = quality[quality.index.notna()]


value = value1.copy()
value.index = pd.to_datetime(value.index, errors='coerce')
value = value[value.index.notna()]

#-------------------------------------------------------------------------------
# Colocar os lookcbacks de cada estrategia

lookback_momentum = 3  #  3 mes
lookback_low_size = 3  #  3 mes
lookback_low_vol = 3 #  3 mes
lookback_quality = 3  #  3 mes
lookback_value = 3  #  3 mes
lookback_ibx = 3  #  3 mes

rebal_time = 1  #  1 mes --> unico para toda a carteira

#-------------------------------------------------------------------------------

closing_price = closing_price2.copy()
value = value2.copy()
low_size = low_size2.copy() 
quality = quality2.copy()
dados_ineg = dados_ineg2.copy()

#-------------------------------------------------------------------------------

dados_ineg = dados_ineg.dropna(axis=1, how='all')

cutoff_date = initial_date
dados_ineg = dados_ineg[dados_ineg.index >= cutoff_date]

## OFENSIVA COM LOOP - PRONTA

In [216]:
returns = pd.DataFrame()

while True:
    rebal = initial_date + pd.DateOffset(months=rebal_time)

    # print(f"Initial: {initial_date}, Rebal: {rebal}, Final: {final_date}")

    if rebal <= final_date:
            
        ineg_date_analysis = initial_date - pd.DateOffset(months=lookback_ibx)
        ineg = dados_ineg[(dados_ineg.index < initial_date) & (dados_ineg.index >= ineg_date_analysis)]
        ineg2 = ineg.copy()
        ineg = ineg.ffill().bfill()

        if not ineg.empty:
                ineg = ineg.iloc[[-1]].T.reset_index()
                ineg.columns = ['ticker', 'neg']
        else:
                initial_date += pd.DateOffset(months=rebal_time)
                continue    # <<<<<< IMPORTANTE AQUI


        #era ticket é o nome da ação e neg é o índice de negociabilidade
        ineg.columns = ['ticker','neg'] if 'neg' not in ineg.columns else ineg.columns


        #no fim essa lista iá significar que as ações investidas estão na lista de boa negociabilidade
        ibx_tickers = ineg.sort_values(by='neg', ascending=False).head(100)['ticker'].tolist()

    # --------------------------------------------------------------

        # OFENSIVA

        #momentum:
        # Define o intervalo para analisar o desempenho passado das ações
        momentum_date_analysis = initial_date - pd.DateOffset(months=lookback_momentum)
        # Filtra os preços dentro do intervalo desejado
        # print(closing_price)
        momentum = closing_price[(closing_price.index < initial_date) & (closing_price.index > momentum_date_analysis)]
        # print(momentum)

        #cálculo do momentum:
        momentum = momentum.pct_change().add(1).cumprod().add(-1)
        # Pega apenas a última linha dos retornos acumulados (mais recente)
        momentum = momentum.iloc[-1]
        # Converte para DataFrame e renomeia as colunas
        momentum = momentum.reset_index()
        momentum.columns = ['ticker', 'momentum'] #Ticker = nome da ação e value = valor da ação
        # print(ibx_tickers)
        momentum = momentum[momentum['ticker'].isin(ibx_tickers)]  #Verifica se a ação está entre as top 100 mais negociadas na bolsa 

        # Ordena da maior performance para a menor
        momentum = momentum.sort_values(by='momentum', ascending=False).reset_index(drop=True)
        # print(momentum)

        # low size em cima de momentum

        momentum_20_pc = momentum['momentum'].quantile(0.80) 
        momentum_20_pc = momentum[momentum['momentum'] >= momentum_20_pc]

        acoes_20_momentum = momentum_20_pc['ticker'].tolist()

        low_size_date_analysis = initial_date - pd.DateOffset(months=lookback_low_size)
        low_size_filtro = low_size[(low_size.index < initial_date) & (low_size.index > low_size_date_analysis)]

        if low_size_filtro.empty:
            # print("Sem dados de low_size para:", initial_date)
            initial_date += pd.DateOffset(months=rebal_time)
            continue


        low_size_filtro = low_size_filtro.iloc[-1]
        low_size_filtro=low_size_filtro.reset_index()
        low_size_filtro.columns = ['ticker','low_size']

        
        low_size_novo = low_size_filtro[low_size_filtro['ticker'].isin(acoes_20_momentum)]
        low_size_novo = low_size_novo.sort_values(by='low_size', ascending=True).reset_index(drop=True)

        lista_low_size = low_size_novo.ticker.tolist()
        
        # analisar se faz sentido essa constatação de winners e loosers ou se é ao contrário

        limite_winners = low_size_novo.low_size.quantile(0.20)

        book_loosers_ofensiva = low_size_novo[low_size_novo['low_size'] > limite_winners]
        book_loosers_ofensiva = book_loosers_ofensiva.ticker.tolist()

        book_winners_ofensiva = low_size_novo[low_size_novo['low_size'] <= limite_winners]   
        book_winners_ofensiva = book_winners_ofensiva.ticker.tolist()


        initial_date = initial_date + pd.DateOffset(months=rebal_time)
    else:
        break

print(book_winners_ofensiva)
print(book_loosers_ofensiva)
print(lista_low_size)



['CVCB3', 'AMER3', 'TEND3', 'CEAB3']
['DIRR3', 'CSMG3', 'POMO4', 'GOAU4', 'STBP3', 'MRFG3', 'NTCO3', 'PSSA3', 'CMIN3', 'CMIG4', 'GGBR4', 'EMBR3', 'RENT3', 'SBSP3', 'SUZB3', 'JBSS3']
['CVCB3', 'AMER3', 'TEND3', 'CEAB3', 'DIRR3', 'CSMG3', 'POMO4', 'GOAU4', 'STBP3', 'MRFG3', 'NTCO3', 'PSSA3', 'CMIN3', 'CMIG4', 'GGBR4', 'EMBR3', 'RENT3', 'SBSP3', 'SUZB3', 'JBSS3']


## DEFENSIVA COM LOOP - TESTE

In [ ]:

returns = pd.DataFrame()

quality_original = quality.copy()
value_original = value.copy()

# print(f"Initial: {initial_date}, Rebal: {rebal}, Final: {final_date}")
while True:
        
        rebal = initial_date + pd.DateOffset(months=rebal_time)
        
        if rebal <= final_date:
        
                ineg_date_analysis = initial_date - pd.DateOffset(months=lookback_ibx)
                ineg = dados_ineg[(dados_ineg.index < initial_date) & (dados_ineg.index >= ineg_date_analysis)]
                ineg2 = ineg.copy()
                ineg = ineg.ffill().bfill()

                if not ineg.empty:
                        ineg = ineg.iloc[[-1]].T.reset_index()
                        ineg.columns = ['ticker', 'neg']
                else:
                        initial_date += pd.DateOffset(months=rebal_time)
                        continue  # <<<<<< IMPORTANTE AQUI

                # print(ineg.shape)
                # print(ineg.columns)

        # era ticket é o nome da ação e neg é o índice de negociabilidade
        # NÃO precisa mais mudar de novo o nome das colunas aqui
        # (já renomeamos antes)



                #era ticket é o nome da ação e neg é o índice de negociabilidade
                ineg.columns = ['ticker','neg'] if 'neg' not in ineg.columns else ineg.columns
                
                #no fim essa lista iá significar que as ações investidas estão na lista de boa negociabilidade
                ibx_tickers = ineg.sort_values(by='neg', ascending=False).head(100)['ticker'].tolist()
                        
                # --------------------------------------------------------------
                low_vol_date_analysis = initial_date - pd.DateOffset(months=lookback_low_vol)
                low_vol_prices = closing_price[(closing_price.index < initial_date) & (closing_price.index > low_vol_date_analysis)]

                # Calcula volatilidade (desvio padrão dos retornos)
                low_vol = low_vol_prices.pct_change().std()  
                low_vol = low_vol.reset_index()
                low_vol.columns = ['ticker', 'low_vol']

                # Filtra para tickers do IBX
                low_vol = low_vol[low_vol['ticker'].isin(ibx_tickers)]
                low_vol = low_vol.sort_values(by='low_vol', ascending=True).reset_index(drop=True)

                low_vol_score = low_vol.copy()
                low_vol_score['score'] = low_vol.index + 1
                
                lista_low_vol = low_vol.ticker.tolist()


                # quality
                # quality
                # print(type(quality.index))
                # print(type(initial_date))
                quality_date_analysis = initial_date - pd.DateOffset(months=lookback_quality)
                quality = quality_original[(quality_original.index < initial_date) & (quality_original.index > quality_date_analysis)]

                if not quality.empty:
                        quality = quality.iloc[[-1]].T.reset_index()
                        quality.columns = ['ticker','quality']
                else:
                        initial_date += pd.DateOffset(months=rebal_time)
                        continue

                # quality_date_analysis = initial_date - pd.DateOffset(months=lookback_quality)
                # quality = quality[(quality.index < initial_date) & (quality.index > quality_date_analysis)]
                # quality = quality.iloc[-1]
                # quality = quality.reset_index()
                # quality.columns = ['ticker','quality']
                quality = quality[quality['ticker'].isin(lista_low_vol)]
                quality = quality.sort_values(by='quality', ascending=False).reset_index(drop=True)

                quality_score = quality.copy()
                quality_score['score'] = quality.index + 1

                quality_score['score_result'] = quality_score['score'] + low_vol_score['score']
                quality_score = quality_score.sort_values(by='score_result', ascending=True).reset_index(drop=True)
                lista_quality = quality.ticker.tolist()


                acoes_25_quality_low_vol = quality_score.score_result.quantile(0.75)
                acoes_25_quality_low_vol = quality_score[quality_score['score_result'] <= acoes_25_quality_low_vol]   
                acoes_25_quality_low_vol = acoes_25_quality_low_vol.ticker.tolist()


                # value

                value_date_analysis = initial_date - pd.DateOffset(months=lookback_value)
                value = value_original[(value_original.index < initial_date) & (value_original.index > value_date_analysis)]
                value = value.iloc[-1]
                value = value.reset_index()
                value.columns = ['ticker','value']
                value = value[value['ticker'].isin(acoes_25_quality_low_vol)]
                value = value.sort_values(by='value', ascending=False).reset_index(drop=True)


                limite_winners = value.value.quantile(0.85)

                book_winners_defensiva = value[value['value'] >= limite_winners]   
                book_winners_defensiva = book_winners_defensiva.ticker.tolist()

                book_loosers_defensiva = value[value['value'] < limite_winners]
                book_loosers_defensiva = book_loosers_defensiva.ticker.tolist()

                print('Initial: ', initial_date)
                print('Rebal: ', rebal)
                initial_date = initial_date + pd.DateOffset(months=rebal_time)
        else:
                break

print(book_loosers_defensiva)
print(book_winners_defensiva)

Initial:  2000-02-24 00:00:00
Rebal:  2000-03-24 00:00:00
Initial:  2000-03-24 00:00:00
Rebal:  2000-04-24 00:00:00
Initial:  2000-04-24 00:00:00
Rebal:  2000-05-24 00:00:00
Initial:  2000-05-24 00:00:00
Rebal:  2000-06-24 00:00:00
Initial:  2000-06-24 00:00:00
Rebal:  2000-07-24 00:00:00
Initial:  2000-07-24 00:00:00
Rebal:  2000-08-24 00:00:00
Initial:  2000-08-24 00:00:00
Rebal:  2000-09-24 00:00:00
Initial:  2000-09-24 00:00:00
Rebal:  2000-10-24 00:00:00
Initial:  2000-10-24 00:00:00
Rebal:  2000-11-24 00:00:00
Initial:  2000-11-24 00:00:00
Rebal:  2000-12-24 00:00:00
Initial:  2000-12-24 00:00:00
Rebal:  2001-01-24 00:00:00
Initial:  2001-01-24 00:00:00
Rebal:  2001-02-24 00:00:00
Initial:  2001-02-24 00:00:00
Rebal:  2001-03-24 00:00:00
Initial:  2001-03-24 00:00:00
Rebal:  2001-04-24 00:00:00
Initial:  2001-04-24 00:00:00
Rebal:  2001-05-24 00:00:00
Initial:  2001-05-24 00:00:00
Rebal:  2001-06-24 00:00:00
Initial:  2001-06-24 00:00:00
Rebal:  2001-07-24 00:00:00
Initial:  2001

In [169]:
teste = []
for x in book_winners_defensiva:
    if x in book_loosers_defensiva:
        teste.append(x)

print(len(teste))
print(len(book_winners_defensiva))
print(len(book_loosers_defensiva))

52
63
63


In [ ]:
returns = pd.DataFrame()
count = 0
while True:
    count+=1

    rebal = initial_date + pd.DateOffset(months=rebal_time)
    
    # print(f"Initial: {initial_date}, Rebal: {rebal}, Final: {final_date}")

    if rebal <= final_date:
         
        ineg_date_analysis = initial_date - pd.DateOffset(months=lookback_ibx)
        ineg = dados_ineg[(dados_ineg.index < initial_date) & (dados_ineg.index >= ineg_date_analysis)]
        ineg = ineg.bfill().ffill()

        if not ineg.empty:
            ineg = ineg.iloc[-1]
        else:
            initial_date += pd.DateOffset(months=rebal_time)
            continue

        ineg = ineg.reset_index()

        #era ticket é o nome da ação e neg é o índice de negociabilidade
        ineg.columns = ['ticker','neg'] if 'neg' not in ineg.columns else ineg.columns

        #no fim essa lista iá significar que as ações investidas estão na lista de boa negociabilidade
        ibx_tickers = ineg.sort_values(by='neg', ascending=False).head(100)['ticker'].tolist()

# --------------------------------------------------------------

        # OFENSIVA

        #momentum:
        # Define o intervalo para analisar o desempenho passado das ações
        momentum_date_analysis = initial_date - pd.DateOffset(months=lookback_momentum)
        # Filtra os preços dentro do intervalo desejado
        momentum = closing_price[(closing_price.index < initial_date) & (closing_price.index > momentum_date_analysis)]
        

        #cálculo do momentum:
        momentum = momentum.pct_change().add(1).cumprod().add(-1)
        # Pega apenas a última linha dos retornos acumulados (mais recente)
        momentum = momentum.iloc[-1]
        # Converte para DataFrame e renomeia as colunas
        momentum = momentum.reset_index()
        momentum.columns = ['ticker', 'momentum'] #Ticker = nome da ação e value = valor da ação
        momentum = momentum[momentum['ticker'].isin(ibx_tickers)]  #Verifica se a ação está entre as top 100 mais negociadas na bolsa 

        # Ordena da maior performance para a menor
        momentum = momentum.sort_values(by='momentum', ascending=False).reset_index(drop=True)

        # low size em cima de momentum

        momentum_20_pc = momentum['momentum'].quantile(0.80) 
        momentum_20_pc = momentum[momentum['momentum'] >= momentum_20_pc]

        acoes_20_momentum = momentum_20_pc['ticker'].tolist()

        low_size_date_analysis = initial_date - pd.DateOffset(months=lookback_low_size)
        low_size_filtro = low_size[(low_size.index < initial_date) & (low_size.index > low_size_date_analysis)]

        if low_size_filtro.empty:
            print("Sem dados de low_size para:", initial_date)
            initial_date += pd.DateOffset(months=rebal_time)
            continue

        low_size_filtro = low_size_filtro.iloc[-1]
        low_size_filtro=low_size_filtro.reset_index()
        low_size_filtro.columns = ['ticker','low_size']

        
        low_size_novo = low_size_filtro[low_size_filtro['ticker'].isin(acoes_20_momentum)]
        low_size_novo = low_size_novo.sort_values(by='low_size', ascending=True).reset_index(drop=True)

        lista_low_size = low_size_novo.ticker.tolist()
        


        book_winners_ofensiva = low_size_novo.ticker.quantile(0.80)
        book_winners_ofensiva = low_size_novo[low_size_novo['low_size'] <= book_winners_ofensiva]
        book_loosers_ofensiva = low_size_novo.ticker.quantile(0.20)
        book_loosers_ofensiva = low_size_novo[low_size_novo['low_size'] >= book_loosers_ofensiva]


        initial_date = initial_date + pd.DateOffset(months=rebal_time)

    else:
        break
# print(rebal)
# print(final_date)
# print(initial_date)
# print(count)


In [ ]:
returns = pd.DataFrame()

while True:

    rebal = initial_date + pd.DateOffset(months=rebal_time)
    
    # print(f"Initial: {initial_date}, Rebal: {rebal}, Final: {final_date}")

    if rebal <= final_date:
         
        ineg_date_analysis = initial_date - pd.DateOffset(months=lookback_ibx)
        ineg = dados_ineg[(dados_ineg.index < initial_date) & (dados_ineg.index >= ineg_date_analysis)]
        ineg = ineg.bfill().ffill()

        if not ineg.empty:
            ineg = ineg.iloc[-1]
        else:
            initial_date += pd.DateOffset(months=rebal_time)
            continue

        ineg = ineg.reset_index()

        #era ticket é o nome da ação e neg é o índice de negociabilidade
        ineg.columns = ['ticker','neg'] if 'neg' not in ineg.columns else ineg.columns

        #no fim essa lista iá significar que as ações investidas estão na lista de boa negociabilidade
        ibx_tickers = ineg.sort_values(by='neg', ascending=False).head(100)['ticker'].tolist()

# --------------------------------------------------------------

        # OFENSIVA

        #momentum:
        # Define o intervalo para analisar o desempenho passado das ações
        momentum_date_analysis = initial_date - pd.DateOffset(months=lookback_momentum)
        # Filtra os preços dentro do intervalo desejado
        momentum = closing_price[(closing_price.index < initial_date) & (closing_price.index > momentum_date_analysis)]
        #cálculo do momentum:
        momentum = momentum.pct_change().add(1).cumprod().add(-1)
        # Pega apenas a última linha dos retornos acumulados (mais recente)
        momentum = momentum.iloc[-1]
        # Converte para DataFrame e renomeia as colunas
        momentum = momentum.reset_index()
        momentum.columns = ['ticker', 'momentum'] #Ticker = nome da ação e value = valor da ação
        momentum = momentum[momentum['ticker'].isin(ibx_tickers)]  #Verifica se a ação está entre as top 100 mais negociadas na bolsa 

        # Ordena da maior performance para a menor
        momentum = momentum.sort_values(by='momentum', ascending=False).reset_index(drop=True)


        # low size em cima de momentum

        momentum_20_pc = momentum['momentum'].quantile(0.80) 
        momentum_20_pc = momentum[momentum['momentum'] >= momentum_20_pc]

        acoes_20_momentum = momentum_20_pc['ticker'].tolist()

        low_size_date_analysis = initial_date - pd.DateOffset(months=lookback_low_size)
        low_size_filtro = low_size[(low_size.index < initial_date) & (low_size.index > low_size_date_analysis)]

        if low_size_filtro.empty:
            print("Sem dados de low_size para:", initial_date)
            initial_date += pd.DateOffset(months=rebal_time)
            continue

        low_size_filtro = low_size_filtro.iloc[-1]
        low_size_filtro=low_size_filtro.reset_index()
        low_size_filtro.columns = ['ticker','low_size']

        
        low_size_novo = low_size_filtro[low_size_filtro['ticker'].isin(acoes_20_momentum)]
        low_size_novo = low_size_novo.sort_values(by='low_size', ascending=True).reset_index(drop=True)

        lista_low_size = low_size_novo.ticker.tolist()
        


        book_winners_ofensiva = low_size_novo.ticker.quantile(0.80)
        book_winners_ofensiva = low_size_novo[low_size_novo['low_size'] <= book_winners_ofensiva]
        book_loosers_ofensiva = low_size_novo.ticker.quantile(0.20)
        book_loosers_ofensiva = low_size_novo[low_size_novo['low_size'] >= book_loosers_ofensiva]


# --------------------------------------------------------------
        # DEFENSIVA

        # low vol

        # Low Vol direto do fechamento
        low_vol_date_analysis = initial_date - pd.DateOffset(months=lookback_low_vol)
        low_vol_prices = closing_price[(closing_price.index < initial_date) & (closing_price.index > low_vol_date_analysis)]

        # Calcula volatilidade (desvio padrão dos retornos)
        low_vol = low_vol_prices.pct_change().std()  # isso retorna uma Series com o ticker como índice
        low_vol = low_vol.reset_index()
        low_vol.columns = ['ticker', 'low_vol']

        # Filtra para tickers do IBX
        low_vol = low_vol[low_vol['ticker'].isin(ibx_tickers)]
        low_vol = low_vol.sort_values(by='low_vol', ascending=True).reset_index(drop=True)

        lista_low_vol = low_vol.ticker.tolist()


        # quality
        quality_date_analysis = initial_date - pd.DateOffset(months=lookback_quality)
        quality = quality[(quality.index < initial_date) & (quality.index > quality_date_analysis)]
        quality = quality.iloc[-1]
        quality = quality.reset_index()
        quality.columns = ['ticker','quality']
        quality = quality[quality['ticker'].isin(ibx_tickers)]
        quality = quality.sort_values(by='quality', ascending=False).reset_index(drop=True)


        lista_quality = quality.ticker.tolist()

        ## necessário implementar aqui os scores de cada ação, utilizando 
        # a soma dos índices da ação das duas listas/dataframes  

        # acoes_25_quality_low_vol = ultimo_df['ticker'].quantile(0.75) # 25%
        # acoes_25_quality_low_vol = ultimo_df[ultimo_df['ticker'] >= acoes_25_quality_low_vol]
        # acoes_25_quality_low_vol = acoes_25_quality_low_vol['ticker'].tolist()

        # value

        value_date_analysis = initial_date - pd.DateOffset(months=lookback_value)
        value = value[(value.index < initial_date) & (value.index > value_date_analysis)]
        value = value.iloc[-1]
        value = value.reset_index()
        value.columns = ['ticker','quality']
        value = value[value['ticker'].isin(ibx_tickers)]
        value = value.sort_values(by='quality', ascending=False).reset_index(drop=True)


        book_winners = momentum.ticker[:int(len(momentum)*0.4)]
        book_loosers = [l for l in list(momentum['ticker']) if l not in list(book_winners)]
        # ou 
        # book_losers = momentum.ticker[int(len(momentum))*0.4:]



        # backtest é o mesmo para winners, losers e universe
        # universe
        backtest_universe = closing_price[(closing_price.index >= initial_date) & (closing_price.index < rebal)]
        backtest_universe = backtest_universe.pct_change().replace(np.nan,0)
        backtest_universe['Retorno'] = backtest_universe.mean(axis=1)
        backtest_universe_ret = backtest_universe['Retorno'][1:]

        # losers
        backtest_loosers = closing_price[(closing_price.index >= initial_date) & (closing_price.index < rebal)]
        backtest_loosers = backtest_loosers[book_loosers].pct_change().replace(np.nan,0)
        backtest_loosers['Retorno'] = backtest_loosers.mean(axis=1)
        backtest_loosers_ret = backtest_loosers['Retorno'][1:]

        # winners
        backtest_winners = closing_price[(closing_price.index >= initial_date) & (closing_price.index < rebal)]
        backtest_winners = backtest_winners[book_winners].pct_change().replace(np.nan,0)
        backtest_winners['Retorno'] = backtest_winners.mean(axis=1)
        backtest_winners_ret = backtest_winners['Retorno'][1:]

        # retorno
        retorno_rebal = pd.DataFrame({
            'Winners': backtest_winners_ret,
            'Universo': backtest_universe_ret,
            'Losers': backtest_loosers_ret,
        })


        returns = pd.concat([returns, retorno_rebal], ignore_index=False)

        initial_date = initial_date + pd.DateOffset(months=rebal_time)

    else:
        break

returns = returns.add(1).cumprod()


qs.reports.full(returns['Winners'],returns['Universo'])


IndexError: single positional indexer is out-of-bounds

## Ponderação de Ofensiva e Defensiva

In [ ]:
# quero fazer um backtest com ponderação para o retorno, 80% do portifólio para defensiva e 20% para ofensiva
# o mesmo para os winners e losers, 80% defensiva e 20% ofensiva
